In [1]:
from GenStudentMixtures.Mixture_Multivariate_Student_Generalized import MMST
from GenStudentMixtures.Multivariate_Student_Generalized import MST

from GenStudentMixtures.GenStudentMixtures import GenStudentMixtures
import numpy as np
from numpy.random import shuffle

from sklearn.cluster import KMeans


## 2D

In [ ]:
# Real Parameters
pi = np.array([.1, .2, .3, .4], dtype=np.float64)
mu = np.array([[0, -6], [0, 0], [0, 6], [-6, 6]], dtype=np.float64)
angle = np.pi / 6
matRot = [[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]]
D = np.array([matRot, matRot, matRot, matRot], dtype=np.float64)
A = np.ones((4, 2), dtype=np.float64)
A = np.array([[2, 3], [1, 2.5], [5, 2], [1.5, 0.9]], dtype=np.float64)
nu = np.array([[1, 3], [1, 3], [1, 3], [1, 3]], dtype=np.float64)

# Sampling
gen_mix = MMST(pi, mu, A, D, nu).sample(1000000)
gen_mix = np.array(sorted(gen_mix, key=lambda e: abs(e.max()) < 10 and abs(e.min()) < 10)[::-1], dtype=np.float64)

# real estimation 
model = KMeans(4, max_iter=3000, tol=1e-5)
gen_mix_init = gen_mix[:500]
model.fit(gen_mix_init)

# Parameters approximation
pi_pred = np.array([(model.labels_ == k).sum() / len(model.labels_) for k in range(4)], dtype=np.float64)
mu_pred = np.array([gen_mix_init[model.labels_ == k].mean(0) for k in range(4)], dtype=np.float64)
D_pred = np.array([np.eye(2), np.eye(2), np.eye(2), np.eye(2)], dtype=np.float64)
A_pred = np.ones(A.shape, dtype=np.float64)
nu_pred = 30 * np.ones(nu.shape, dtype=np.float64)

# Shuffle the samples
shuffle(gen_mix)

# Learning rates
gam = (1-10e-10)*np.array([k for k in range(1, len(gen_mix) + 1)]) ** (-6/10)

## 3D

In [2]:
# Real Parameters
pi = np.array([.3, .5, .2], dtype=np.float64)
mu = np.array([[0, -6, 0], [0, 0, 0], [0, 6, 0]], dtype=np.float64)
angle = np.pi / 6
matRot = [[1, 0, 0], [0, np.cos(angle), -np.sin(angle)], [0, np.sin(angle), np.cos(angle)]]
D = np.array([matRot, matRot, matRot], dtype=np.float64)
A = np.ones((3, 3), dtype=np.float64)
nu = np.array([[1, 3, 5], [1, 3, 5], [1, 3, 5]], dtype=np.float64)

# Sampling
gen_mix = MMST(pi, mu, A, D, nu).sample(1000000)
gen_mix = np.array(sorted(gen_mix, key=lambda e: abs(e.max()) < 10 and abs(e.min()) < 10)[::-1], dtype=np.float64)

# KMeans to approximate parameters
model = KMeans(3, max_iter=3000, tol=1e-5)
gen_mix_init = gen_mix[:500]
model.fit(gen_mix_init)

# Parameters approximation
pi_pred = np.array([(model.labels_ == k).sum() / len(model.labels_) for k in range(3)], dtype=np.float64)
mu_pred = np.array([gen_mix_init[model.labels_ == k].mean(0) for k in range(3)], dtype=np.float64)
D_pred = np.array([np.eye(3), np.eye(3), np.eye(3)], dtype=np.float64)
A_pred = np.ones(A.shape, dtype=np.float64)
nu_pred = np.ones(nu.shape, dtype=np.float64)

# Shuffle the samples
shuffle(gen_mix)

# Learning rates
gam = (1-10e-10)*np.array([k for k in range(1, len(gen_mix) + 1)]) ** (-6/10)

In [3]:
model = GenStudentMixtures(pi_pred, mu_pred, A_pred, D_pred, nu_pred)

In [4]:
model.fit(gen_mix[1:], gam, mini_batch=50)

  0%|                                                 | 0/19999 [00:00<?, ?it/s]

[0.474 0.206 0.32 ]


  3%|▉                                    | 502/19999 [06:26<4:44:39,  1.14it/s]

[0.51033727 0.20180905 0.28785368]


  3%|▉                                    | 506/19999 [06:34<4:13:17,  1.28it/s]


KeyboardInterrupt: 